In [ ]:
from access import Access, weights, Datasets
import logging
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [81]:
lima = gpd.read_file("./Lima_pop.gpkg").to_crs({'init': 'epsg:32718'})
cs = gpd.read_file("cs_ability.gpkg").to_crs({'init':'epsg:32718'})

/usr/lib/python3.10/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/lib/python3.10/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [85]:
lima.head()

,OBJECTID,IDMANZANA,CODDPTO,CODPROV,CODDIST,CODZONA,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,IDCCPP,NOMCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,vivienda,hogar,poblacion,casa,geometry
0,341229.0,150101013000450,15,01,01,013,00,045,0,150101,1501010001,LIMA,LIMA,LIMA,LIMA,52.0,41.0,142.0,100.0,MULTIPOINT (275183.114 8666686.203)
1,341230.0,150101013000410,15,01,01,013,00,041,0,150101,1501010001,LIMA,LIMA,LIMA,LIMA,56.0,58.0,204.0,100.0,MULTIPOINT (275075.796 8666642.137)
2,341232.0,150101013000370,15,01,01,013,00,037,0,150101,1501010001,LIMA,LIMA,LIMA,LIMA,14.0,14.0,61.0,100.0,MULTIPOINT (274969.901 8666672.002)
3,341233.0,150101013000340,15,01,01,013,00,034,0,150101,1501010001,LIMA,LIMA,LIMA,LIMA,30.0,31.0,101.0,100.0,MULTIPOINT (274934.517 8666670.117)
4,341234.0,150101013000330,15,01,01,013,00,033,0,150101,1501010001,LIMA,LIMA,LIMA,LIMA,10.0,13.0,47.0,100.0,MULTIPOINT (274906.752 8666657.810)


In [86]:
cs_access = Access(
    demand_df = lima,
    demand_index = 'UBIGEO',
    demand_value = 'poblacion',
    supply_df    = cs,
    supply_index = 'codigo')

access INFO     :: Warning: A supply value was not provided, so a default
                             supply value of 1 was created in the column named "value".
                             Note that without a supply value, you cannot use any of the
                             floating catchment area methods.


In [87]:
cs_access.create_euclidean_distance(name = 'cost', threshold=100000, centroid_d=True, centroid_o=True)

In [88]:
cost_cs = cs_access.cost_df
cost_cs["cost"] = cs_access.cost_df["cost"]/1000

In [90]:
cost_cs.head()

,cost,origin,dest
0,14.786563,150101,00005819
1,14.814577,150101,00005819
2,14.770375,150101,00005819
3,14.767518,150101,00005819
4,14.776071,150101,00005819


In [92]:
A = Access(demand_df            = lima,
           demand_index         = 'UBIGEO',
           demand_value         = 'poblacion',
           supply_df            = cs,
           supply_index         = 'codigo',
           supply_value         = ['Medico','Odontologo'],
           cost_df              = cost_cs,
           cost_origin          = 'origin',
           cost_dest            = 'dest',
           cost_name            = 'cost',
           neighbor_cost_df     = cost_cs,
           neighbor_cost_origin = 'origin',
           neighbor_cost_dest   = 'dest',
           neighbor_cost_name   = 'cost')

In [93]:
A.create_euclidean_distance(threshold = 100000, centroid_o = True, centroid_d = True)

MemoryError: Unable to allocate 780. GiB for an array with shape (104686065711,) and data type int64

In [ ]:
A.cost_df

In [49]:
fn30 = weights.step_fn({10 : 1, 20 : 0.68, 30 : 0.22})
fn60 = weights.step_fn({20 : 1, 40 : 0.68, 60 : 0.22})

In [50]:
gaussian = weights.gaussian(20)

In [51]:
gravity = weights.gravity(scale = 60, alpha = -1)

access INFO     :: Using default cost, cost, for 3sfca.


KeyError: 'pop_W'